In [ ]:
import pandas as pd
from pandas import json_normalize
import requests
import json
import snowflake.connector

from dotenv import load_dotenv
import os
import http.client
import urllib.parse


from googletrans import Translator
import string
import re

from sqlalchemy import create_engine

/home/nxie/code/nikki099/de_linkedin_etl_project/.venv/lib/python3.12/site-packages/snowflake/connector/options.py:104: UserWarning: You have an incompatible version of 'pyarrow' installed (20.0.0), please install a version that adheres to: 'pyarrow<19.0.0; extra == "pandas"'
  warn_incompatible_dep(


In [ ]:
load_dotenv()
rapidapi_key = os.getenv('RAPIDAPI_KEY')
rapidapi_host = "linkedin-job-search-api.p.rapidapi.com"
snowflake_user = os.getenv('SNOWFLAKE_USER')
account = os.getenv('SNOWFLAKE_ACCOUNT')
snowflake_password = os.getenv('SNOWFLAKE_PASSWORD')

'NIKKILW2025'

In [4]:
def extract_linkedin_job_data():

    headers = {
        'x-rapidapi-key': rapidapi_key,
        'x-rapidapi-host': rapidapi_host
    }

    location = "Australia"
    limit = 100
    offset = 0
    titles = ["Data Engineer" , "Data Scientist", "Data Analyst"]

    df_daily_all = pd.DataFrame()

    for title_filter in titles:

        #URL code the title filter and location filter
        title_encoded=urllib.parse.quote(title_filter)
        location_encoded=urllib.parse.quote(location)

        #API endpoint 7day data query
        #base_url = f"/active-jb-7d?limit={limit}&offset={offset}&title_filter={title_encoded}&location_filter={location_encoded}"

        #API endpoint last 24 hour data query
        base_url = f"/active-jb-24h?limit={limit}&offset={offset}&title_filter={title_encoded}&location_filter={location_encoded}"
        url =  f"https://{rapidapi_host}{base_url}"


        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json()
            # Convert the JSON data to a DataFrame
            df_daily = json_normalize(data)
            df_daily['job_category'] = title_filter
            print(title_filter, df_daily.shape)
            df_daily_all = pd.concat([df_daily_all, df_daily], ignore_index=True)


        else:
            print(f"Error: {response.status_code}")

    return df_daily_all

df_daily_all = extract_linkedin_job_data()
# df_daily_all

Data Engineer (13, 51)
Data Scientist (2, 45)
Data Analyst (5, 45)


In [5]:
# df_daily_all = pd.read_csv('linkedin_jobs_daily.csv')
df_daily_all.head()

,id,date_posted,date_created,title,organization,organization_url,date_validthrough,locations_raw,location_type,location_requirements_raw,...,seniority,directapply,linkedin_org_slug,salary_raw.@type,salary_raw.currency,salary_raw.value.@type,salary_raw.value.minValue,salary_raw.value.maxValue,salary_raw.value.unitText,job_category
0,1649915521,2025-05-12T04:48:04,2025-05-12T05:03:10.476138,Data Engineer,Talent,https://www.linkedin.com/company/talent-intern...,2025-06-11T04:48:04,"[{'@type': 'Place', 'address': {'@type': 'Post...",None,None,...,Mid-Senior level,False,talent-international,NaN,NaN,NaN,NaN,NaN,NaN,Data Engineer
1,1649915481,2025-05-12T04:48:04,2025-05-12T05:03:09.672115,Data Engineer,Talent,https://www.linkedin.com/company/talent-intern...,2025-06-11T04:48:04,"[{'@type': 'Place', 'address': {'@type': 'Post...",None,None,...,Mid-Senior level,False,talent-international,NaN,NaN,NaN,NaN,NaN,NaN,Data Engineer
2,1649812350,2025-05-12T04:20:29,2025-05-12T04:28:46.464171,Lead Data Engineer,tekFinder,https://www.linkedin.com/company/tekfinder,2025-06-11T04:20:29,"[{'@type': 'Place', 'address': {'@type': 'Post...",None,None,...,Руководитель среднего звена,True,tekfinder,NaN,NaN,NaN,NaN,NaN,NaN,Data Engineer
3,1649811042,2025-05-12T04:18:48,2025-05-12T04:27:06.301682,Staff Data Engineer,Blinq,https://www.linkedin.com/company/blinq-me,2025-06-11T04:18:48,"[{'@type': 'Place', 'address': {'@type': 'Post...",None,None,...,Geçerli Değil,True,blinq-me,NaN,NaN,NaN,NaN,NaN,NaN,Data Engineer
4,1649682513,2025-05-12T03:06:48,2025-05-12T03:33:21.830874,Data Engineer (Mid-level),Airtree,https://www.linkedin.com/company/airtree-ventures,2025-06-11T03:06:02,"[{'@type': 'Place', 'address': {'@type': 'Post...",None,None,...,Entry level,True,airtree-ventures,NaN,NaN,NaN,NaN,NaN,NaN,Data Engineer


In [6]:
def get_clean_data_jobs(df_daily_all):
     pattern=re.compile(r'\bData Engineer\b|\bData Scientist\b|\bData Analyst\b' , re.IGNORECASE)
     df_daily_all = df_daily_all[df_daily_all['title'].str.contains(pattern)]
     return df_daily_all

df_daily_all = get_clean_data_jobs(df_daily_all)
df_daily_all.shape

(19, 51)

In [7]:
df_daily_all.to_csv('linkedin_jobs_daily.csv', index=False)

In [8]:
df_daily_all = pd.read_csv('linkedin_jobs_daily.csv')
df_daily_all.shape

(19, 51)

In [9]:
def update_columns(df_daily_all):
    df_daily_all.columns = df_daily_all.columns.str.upper()
    df_daily_all = df_daily_all[['ID', 'DATE_POSTED', 'DATE_CREATED', 'TITLE', 'JOB_CATEGORY',
       'ORGANIZATION', 'ORGANIZATION_URL', 'DATE_VALIDTHROUGH', 'LOCATIONS_RAW',
       'LOCATION_TYPE', 'LOCATION_REQUIREMENTS_RAW', 'EMPLOYMENT_TYPE', 'URL',
       'SOURCE_TYPE', 'SOURCE', 'SOURCE_DOMAIN', 'ORGANIZATION_LOGO',
       'CITIES_DERIVED', 'REGIONS_DERIVED', 'COUNTRIES_DERIVED',
       'LOCATIONS_DERIVED', 'TIMEZONES_DERIVED', 'LATS_DERIVED',
       'LNGS_DERIVED', 'REMOTE_DERIVED', 'RECRUITER_NAME', 'RECRUITER_TITLE',
       'RECRUITER_URL', 'LINKEDIN_ORG_EMPLOYEES', 'LINKEDIN_ORG_URL',
       'LINKEDIN_ORG_SIZE', 'LINKEDIN_ORG_SLOGAN', 'LINKEDIN_ORG_INDUSTRY',
       'LINKEDIN_ORG_FOLLOWERS', 'LINKEDIN_ORG_HEADQUARTERS',
       'LINKEDIN_ORG_TYPE', 'LINKEDIN_ORG_FOUNDEDDATE',
       'LINKEDIN_ORG_SPECIALTIES', 'LINKEDIN_ORG_LOCATIONS',
       'LINKEDIN_ORG_DESCRIPTION', 'LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED',
       'SENIORITY', 'DIRECTAPPLY', 'LINKEDIN_ORG_SLUG']]
    return df_daily_all

df_daily_all = update_columns(df_daily_all)
df_daily_all.head(2)

,ID,DATE_POSTED,DATE_CREATED,TITLE,JOB_CATEGORY,ORGANIZATION,ORGANIZATION_URL,DATE_VALIDTHROUGH,LOCATIONS_RAW,LOCATION_TYPE,...,LINKEDIN_ORG_HEADQUARTERS,LINKEDIN_ORG_TYPE,LINKEDIN_ORG_FOUNDEDDATE,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG
0,1649915521,2025-05-12T04:48:04,2025-05-12T05:03:10.476138,Data Engineer,Data Engineer,Talent,https://www.linkedin.com/company/talent-intern...,2025-06-11T04:48:04,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"Sydney, NSW",Privately Held,1995.0,"['IT and Technology Recruitment Specialist', '...","['Level 12, 2 Park Street, Sydney, NSW 2000, A...",Talent is a global recruitment specialist whic...,True,Mid-Senior level,False,talent-international
1,1649915481,2025-05-12T04:48:04,2025-05-12T05:03:09.672115,Data Engineer,Data Engineer,Talent,https://www.linkedin.com/company/talent-intern...,2025-06-11T04:48:04,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"Sydney, NSW",Privately Held,1995.0,"['IT and Technology Recruitment Specialist', '...","['Level 12, 2 Park Street, Sydney, NSW 2000, A...",Talent is a global recruitment specialist whic...,True,Mid-Senior level,False,talent-international


### Process the df_daily_all data 


In [10]:
def extract_job_date(date_created):
    #extract the job_date from job_created
    df_daily_all['job_date'] = pd.to_datetime(df_daily_all['DATE_CREATED']).dt.date
    return df_daily_all['job_date']

#Apply the function to the DataFrame
df_daily_all['job_date'] = extract_job_date(df_daily_all['DATE_CREATED'])
df_daily_all['job_date'][:5]


0    2025-05-12
1    2025-05-12
2    2025-05-12
3    2025-05-12
4    2025-05-12
Name: job_date, dtype: object

In [11]:
#Extrac the job city and state from the LOCATIONS_RAW field

def extract_city(list):
    """
    Extracts the city from the given text using regex.
    """
    # Regex pattern to match the city in the LOCATIONS_RAW field
    text = str(list)
    city_pattern = r"'addressLocality':\s*'(.*)',\s'addressRegion':"
    match = re.search(city_pattern, text)
    if match:
        city = match.group(1)
        if 'sidney' in city.lower() or 'sídney' in city.lower() or '悉尼' in city.lower(): #fix wrong city name
            return "Sydney"
        return city
    else:
        return None



def extract_state(list):
    """
    Extracts the state from the given text using regex.
    """
    # Regex pattern to match the region/state in the LOCATIONS_RAW field
    text = str(list)
    state_pattern = r"'addressRegion':\s*(.*)',\s'streetAddress'"
    match = re.search(state_pattern, text)
    if match:
        state = match.group(1)
        state= state.replace("'", "").strip()
        return state
    else:
        return None


#Extract city and state from Locations Raw
df_daily_all['city'] = df_daily_all['LOCATIONS_RAW'].apply(extract_city)


df_daily_all['state'] = df_daily_all['LOCATIONS_RAW'].apply(extract_state)



df_daily_all.head()

,ID,DATE_POSTED,DATE_CREATED,TITLE,JOB_CATEGORY,ORGANIZATION,ORGANIZATION_URL,DATE_VALIDTHROUGH,LOCATIONS_RAW,LOCATION_TYPE,...,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG,job_date,city,state
0,1649915521,2025-05-12T04:48:04,2025-05-12T05:03:10.476138,Data Engineer,Data Engineer,Talent,https://www.linkedin.com/company/talent-intern...,2025-06-11T04:48:04,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"['IT and Technology Recruitment Specialist', '...","['Level 12, 2 Park Street, Sydney, NSW 2000, A...",Talent is a global recruitment specialist whic...,True,Mid-Senior level,False,talent-international,2025-05-12,Brisbane,QLD
1,1649915481,2025-05-12T04:48:04,2025-05-12T05:03:09.672115,Data Engineer,Data Engineer,Talent,https://www.linkedin.com/company/talent-intern...,2025-06-11T04:48:04,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"['IT and Technology Recruitment Specialist', '...","['Level 12, 2 Park Street, Sydney, NSW 2000, A...",Talent is a global recruitment specialist whic...,True,Mid-Senior level,False,talent-international,2025-05-12,Brisbane,QLD
2,1649812350,2025-05-12T04:20:29,2025-05-12T04:28:46.464171,Lead Data Engineer,Data Engineer,tekFinder,https://www.linkedin.com/company/tekfinder,2025-06-11T04:20:29,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"['technology recruitment', 'digital technology...","['Sydney CBD, Sydney, NSW 2000, AU']",Hello there. We’re tekFinder.\n\nAnd we’re a t...,True,Руководитель среднего звена,True,tekfinder,2025-05-12,Сидней,NSW
3,1649811042,2025-05-12T04:18:48,2025-05-12T04:27:06.301682,Staff Data Engineer,Data Engineer,Blinq,https://www.linkedin.com/company/blinq-me,2025-06-11T04:18:48,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,[''],"['4 Bank Pl, Level 3, Melbourne, Victoria 3000...","At Blinq, our vision is to power the start of ...",False,Geçerli Değil,True,blinq-me,2025-05-12,Melbourne,VIC
4,1649682513,2025-05-12T03:06:48,2025-05-12T03:33:21.830874,Data Engineer (Mid-level),Data Engineer,Airtree,https://www.linkedin.com/company/airtree-ventures,2025-06-11T03:06:02,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,[''],"['131 Devonshire St, Surry Hills, New South Wa...",Airtree is a venture capital firm backing Auss...,False,Entry level,True,airtree-ventures,2025-05-12,Sydney,NSW


In [12]:
#Extract the employment type from the EMPLOYMENT_TYPE field

def extract_employment_type(df_daily_all):
    """
    Extracts the employment type from the given text using regex.
    """
    df_daily_all['employment_type'] = (
        df_daily_all['EMPLOYMENT_TYPE']
        .astype(str)
        .str.replace(r"[\[\]']", '', regex=True)
        .str.strip()
    )

    df_daily_all.drop(columns=['EMPLOYMENT_TYPE'], inplace=True)

    return df_daily_all['employment_type']


df_daily_all['employment_type'] = extract_employment_type(df_daily_all)
df_daily_all['employment_type'].tail()

14    FULL_TIME
15    FULL_TIME
16    FULL_TIME
17    FULL_TIME
18    FULL_TIME
Name: employment_type, dtype: object

In [13]:
#Extract the employment size
def extract_employee_size(LINKEDIN_ORG_SIZE):
    """
    Extracts the employee size from the given text using regex.
    """
    df_daily_all['org_size'] = (
        df_daily_all['LINKEDIN_ORG_SIZE']
        .astype(str)
        .str.replace(r"employees", '', regex=True)
        .str.strip()
    )

    return df_daily_all['org_size']


df_daily_all['org_size'] = extract_employee_size(df_daily_all['LINKEDIN_ORG_SIZE'])
df_daily_all['org_size'].tail()

14         2-10
15    501-1,000
16      201-500
17        11-50
18      201-500
Name: org_size, dtype: object

In [14]:
df_daily_all.columns = df_daily_all.columns.str.upper()
df_daily_all.head(2)

,ID,DATE_POSTED,DATE_CREATED,TITLE,JOB_CATEGORY,ORGANIZATION,ORGANIZATION_URL,DATE_VALIDTHROUGH,LOCATIONS_RAW,LOCATION_TYPE,...,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG,JOB_DATE,CITY,STATE,EMPLOYMENT_TYPE,ORG_SIZE
0,1649915521,2025-05-12T04:48:04,2025-05-12T05:03:10.476138,Data Engineer,Data Engineer,Talent,https://www.linkedin.com/company/talent-intern...,2025-06-11T04:48:04,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,Talent is a global recruitment specialist whic...,True,Mid-Senior level,False,talent-international,2025-05-12,Brisbane,QLD,CONTRACTOR,201-500
1,1649915481,2025-05-12T04:48:04,2025-05-12T05:03:09.672115,Data Engineer,Data Engineer,Talent,https://www.linkedin.com/company/talent-intern...,2025-06-11T04:48:04,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,Talent is a global recruitment specialist whic...,True,Mid-Senior level,False,talent-international,2025-05-12,Brisbane,QLD,CONTRACTOR,201-500


In [15]:
#Only keep the relevant columns

df_daily_all = df_daily_all[['ID', 'TITLE', 'JOB_CATEGORY',
       'JOB_DATE', 'CITY', 'STATE', 'EMPLOYMENT_TYPE' ,
       'ORGANIZATION', 'ORGANIZATION_URL', 'URL',
       'SOURCE_TYPE', 'SOURCE', 'SOURCE_DOMAIN',
       'ORGANIZATION_LOGO', 'REMOTE_DERIVED', 'RECRUITER_NAME', 'RECRUITER_TITLE',
       'RECRUITER_URL', 'LINKEDIN_ORG_URL',
       'ORG_SIZE', 'LINKEDIN_ORG_INDUSTRY',
       'LINKEDIN_ORG_HEADQUARTERS',
       'LINKEDIN_ORG_TYPE', 'LINKEDIN_ORG_FOUNDEDDATE',
       'LINKEDIN_ORG_SPECIALTIES', 'LINKEDIN_ORG_LOCATIONS',
       'LINKEDIN_ORG_DESCRIPTION','LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED',
       'SENIORITY', 'DIRECTAPPLY',
       'LINKEDIN_ORG_SLUG']]


df_daily_all.head()

,ID,TITLE,JOB_CATEGORY,JOB_DATE,CITY,STATE,EMPLOYMENT_TYPE,ORGANIZATION,ORGANIZATION_URL,URL,...,LINKEDIN_ORG_HEADQUARTERS,LINKEDIN_ORG_TYPE,LINKEDIN_ORG_FOUNDEDDATE,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG
0,1649915521,Data Engineer,Data Engineer,2025-05-12,Brisbane,QLD,CONTRACTOR,Talent,https://www.linkedin.com/company/talent-intern...,https://au.linkedin.com/jobs/view/data-enginee...,...,"Sydney, NSW",Privately Held,1995.0,"['IT and Technology Recruitment Specialist', '...","['Level 12, 2 Park Street, Sydney, NSW 2000, A...",Talent is a global recruitment specialist whic...,True,Mid-Senior level,False,talent-international
1,1649915481,Data Engineer,Data Engineer,2025-05-12,Brisbane,QLD,CONTRACTOR,Talent,https://www.linkedin.com/company/talent-intern...,https://au.linkedin.com/jobs/view/data-enginee...,...,"Sydney, NSW",Privately Held,1995.0,"['IT and Technology Recruitment Specialist', '...","['Level 12, 2 Park Street, Sydney, NSW 2000, A...",Talent is a global recruitment specialist whic...,True,Mid-Senior level,False,talent-international
2,1649812350,Lead Data Engineer,Data Engineer,2025-05-12,Сидней,NSW,FULL_TIME,tekFinder,https://www.linkedin.com/company/tekfinder,https://au.linkedin.com/jobs/view/lead-data-en...,...,"Sydney, NSW",Public Company,2018.0,"['technology recruitment', 'digital technology...","['Sydney CBD, Sydney, NSW 2000, AU']",Hello there. We’re tekFinder.\n\nAnd we’re a t...,True,Руководитель среднего звена,True,tekfinder
3,1649811042,Staff Data Engineer,Data Engineer,2025-05-12,Melbourne,VIC,FULL_TIME,Blinq,https://www.linkedin.com/company/blinq-me,https://au.linkedin.com/jobs/view/staff-data-e...,...,"Melbourne, Victoria",Privately Held,NaN,[''],"['4 Bank Pl, Level 3, Melbourne, Victoria 3000...","At Blinq, our vision is to power the start of ...",False,Geçerli Değil,True,blinq-me
4,1649682513,Data Engineer (Mid-level),Data Engineer,2025-05-12,Sydney,NSW,FULL_TIME,Airtree,https://www.linkedin.com/company/airtree-ventures,https://au.linkedin.com/jobs/view/data-enginee...,...,"Surry Hills, New South Wales",Public Company,2014.0,[''],"['131 Devonshire St, Surry Hills, New South Wa...",Airtree is a venture capital firm backing Auss...,False,Entry level,True,airtree-ventures


## Connect to  Snowflake database for raw data initial processing

In [ ]:
#Establish a connection to Snowflake

def connect_to_snowflake():
    try:

        conn = snowflake.connector.connect(
            user=snowflake_user,
            password=snowflake_password,
            account=account,
            warehouse="SNOWFLAKE_LEARNING_WH",
            database="linkedin_db",
            schema="linkedin_raw"
        )
        print("Connection to Snowflake established successfully.")
        return conn
    except Exception as e:
        print(f"Error connecting to Snowflake: {e}")
        return None

conn = connect_to_snowflake()

Connection to Snowflake established successfully.


In [17]:
#query the raw data and narrow down to DE, DS and DA roles
#This data is filtered by relevant roles, and translated into English (but no city, state and seniority fix)
def query_existing_job_data(conn):
    query = """
        SELECT * FROM LINKEDIN_JOB_API_CLEANED_DATA
        WHERE (
            lower(TITLE) LIKE '%data engineer%'
            OR lower(TITLE) LIKE '%data scientist%'
            OR lower(TITLE) LIKE '%data analyst%'
            )
    """

    df = pd.read_sql(query, conn)
    print(df.shape)
    return df

df = query_existing_job_data(conn)
df.head()

/tmp/ipykernel_29117/824833193.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


(440, 31)


,ID,TITLE,JOB_CATEGORY,JOB_DATE,CITY,STATE,EMPLOYMENT_TYPE,ORGANIZATION,ORGANIZATION_URL,URL,...,LINKEDIN_ORG_HEADQUARTERS,LINKEDIN_ORG_TYPE,LINKEDIN_ORG_FOUNDEDDATE,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG
0,1599037955,Senior Data Engineer,Data Engineer,2025-04-28,Greater Sydney Area,NA,FULL_TIME,One51 | Data & Analytics Consultancy,https://www.linkedin.com/company/one51consulting,https://au.linkedin.com/jobs/view/senior-data-...,...,"Sydney, NSW",Privately Held,2020.0,"['Business Intelligence', 'Data Warehousing', ...","['333 George Street, Level 13, Sydney, NSW 200...",Drawing on a wealth of expertise and a deep un...,False,Mid-Senior level,False,one51consulting
1,1598878880,Senior Data Engineer,Data Engineer,2025-04-28,Brisbane,QLD,CONTRACTOR,Data#3,https://www.linkedin.com/company/data3,https://au.linkedin.com/jobs/view/senior-data-...,...,"Toowong, Queensland",Public Company,1977.0,"['Cloud Solutions', 'Mobility Solutions', 'Sec...","['555 Coronation Dr, Toowong, Queensland 4066,...","Data#3 Limited (DTL), is focused on helping cu...",False,Medium-high level,False,data3
2,1598880069,Data Engineer,Data Engineer,2025-04-28,Sydney,NSW,CONTRACTOR,Whizdom,https://www.linkedin.com/company/whizdom-recru...,https://au.linkedin.com/jobs/view/data-enginee...,...,"Canberra, Australian Capital Territory",Privately Held,2006.0,"['IT Recruitment', 'Recruitment for Government...","['28-34 Thynne St, Unit 7, Canberra, Australia...","Established in 2006, Whizdom is an Australian ...",True,Middle level,True,whizdom-recruitment
3,1598878894,Senior Data Engineer and Business Intelligence...,Data Engineer,2025-04-28,Moreton Bay,QLD,FULL_TIME,University of the Sunshine Coast,https://www.linkedin.com/school/university-of-...,https://au.linkedin.com/jobs/view/senior-data-...,...,None,None,NaN,None,None,None,None,Not Applicable,False,None
4,1598880083,Senior Data Engineer and Business Intelligence...,Data Engineer,2025-04-28,Sunshine Coast,QLD,FULL_TIME,University of the Sunshine Coast,https://www.linkedin.com/school/university-of-...,https://au.linkedin.com/jobs/view/senior-data-...,...,None,None,NaN,None,None,None,None,Not Applicable,False,None


In [18]:
df.to_csv('api_cleaned_job_id_snowflake.csv', index=False)

In [19]:
list_daily = df_daily_all.columns.tolist()
list_df = df.columns.tolist()
list_daily ==  list_df

True

In [20]:
#Check the Job ID from df and only keep those new jobs based on the Job IDs
def keep_new_jobs():
    existing_job_ids = df.ID.unique().tolist()

    df_new_jobs = df_daily_all[~(df_daily_all['ID'].isin(existing_job_ids))].reset_index(drop=True)
    print(f'{df_new_jobs.shape[0]} new jobs ready to load to Snowflake')
    return df_new_jobs

df_new_jobs = keep_new_jobs()
df_new_jobs.tail()

7 new jobs ready to load to Snowflake


,ID,TITLE,JOB_CATEGORY,JOB_DATE,CITY,STATE,EMPLOYMENT_TYPE,ORGANIZATION,ORGANIZATION_URL,URL,...,LINKEDIN_ORG_HEADQUARTERS,LINKEDIN_ORG_TYPE,LINKEDIN_ORG_FOUNDEDDATE,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG
2,1649812350,Lead Data Engineer,Data Engineer,2025-05-12,Сидней,NSW,FULL_TIME,tekFinder,https://www.linkedin.com/company/tekfinder,https://au.linkedin.com/jobs/view/lead-data-en...,...,"Sydney, NSW",Public Company,2018.0,"['technology recruitment', 'digital technology...","['Sydney CBD, Sydney, NSW 2000, AU']",Hello there. We’re tekFinder.\n\nAnd we’re a t...,True,Руководитель среднего звена,True,tekfinder
3,1649811042,Staff Data Engineer,Data Engineer,2025-05-12,Melbourne,VIC,FULL_TIME,Blinq,https://www.linkedin.com/company/blinq-me,https://au.linkedin.com/jobs/view/staff-data-e...,...,"Melbourne, Victoria",Privately Held,NaN,[''],"['4 Bank Pl, Level 3, Melbourne, Victoria 3000...","At Blinq, our vision is to power the start of ...",False,Geçerli Değil,True,blinq-me
4,1649914883,Senior Data Scientist,Data Scientist,2025-05-12,Sydney,NSW,FULL_TIME,Commonwealth Bank,https://www.linkedin.com/company/commonwealthbank,https://au.linkedin.com/jobs/view/senior-data-...,...,"Sydney, NSW",Public Company,NaN,[''],"['., Sydney, NSW 2000, AU']",Australia’s leading provider of financial serv...,False,Mid-Senior level,False,commonwealthbank
5,1649920748,Expressions of Interest - Senior Finance Data ...,Data Analyst,2025-05-12,Sydney,NSW,FULL_TIME,Propel Recruitment,https://www.linkedin.com/company/propel-recrui...,https://au.linkedin.com/jobs/view/expressions-...,...,"Sydney, NSW",Privately Held,2021.0,[''],"['Sydney, NSW \t2000, AU']",Follow us for job postings & industry insights...,True,Mid-Senior level,False,propel-recruitment-australia
6,1649975915,Graduate Program 2026 - Data Analyst,Data Analyst,2025-05-12,Perth,WA,FULL_TIME,Synergy (Electricity Generation and Retail Cor...,https://www.linkedin.com/company/synergy-retai...,https://au.linkedin.com/jobs/view/graduate-pro...,...,"Perth, Western Australia",Government Agency,2006.0,['Energy Retailer and Energy Generation'],"['152-158 St Georges Tce, Perth, Western Austr...",As Western Australia’s leading energy generato...,False,Internship,False,synergy-retail-electricity-corporation-


In [21]:
#Translate new job's business name, city, state and seniority to English

translator = Translator()

def translate_text(text, target_language='en'):
    try:
        if not text or pd.isna(text):
            return 'NA'
        translated_text = translator.translate(str(text), dest=target_language)
        return translated_text.text
    except Exception as e:
        print(f"Error translating text: {e} (Text: {text})")
        return text

for col in ['CITY', 'STATE', 'ORGANIZATION', 'SENIORITY']:
    unique_values = df_new_jobs[col].dropna().unique()
    translation_map = {val: translate_text(val, target_language='en') for val in unique_values}
    df_new_jobs[col] = df_new_jobs[col].map(translation_map).fillna('NA')

In [22]:
df_new_jobs['CITY'].unique()

array(['Brisbane', 'Sydney', 'Melbourne', 'Perth'], dtype=object)

In [23]:
df_new_jobs['STATE'].unique()

array(['QLD', 'NSW', 'VIC', 'Of'], dtype=object)

In [ ]:

def load_to_snowflake(df_new_jobs):
    # Create a Snowflake connection engine
    engine = create_engine(
        'snowflake://{user}:{password}@{account}/{database}/{schema}?warehouse={warehouse}'.format(
            user=snowflake_user,
            password=snowflake_password,
            account=account,
            warehouse="SNOWFLAKE_LEARNING_WH",
            database="linkedin_db",
            schema="linkedin_raw"
        )
    )

    table_name = "linkedin_job_api_cleaned_data"

    df_new_jobs.to_sql(
        name=table_name,
        con=engine,
        if_exists='append', #append data
        index=False
    )

    print(f"Data loaded to Snowflake table {table_name} successfully.")


load_to_snowflake(df_new_jobs)